# 🤖 Fine-Tuning do Phi-3 Mini
**Projeto:** Modelo de IA especializado em ensino de programação e Vibe Coding
**Modelo base:** Microsoft Phi-3 Mini 4K Instruct
**Técnica:** LoRA (Low-Rank Adaptation)
**Ambiente:** Google Colab (GPU T4 gratuita)

---

## Estrutura do Projeto
1. Instalação de dependências
2. Carregamento do modelo base
3. Preparação do dataset
4. Configuração do LoRA
5. Treinamento (Fine-Tuning)
6. Salvamento do modelo
7. Teste e comparação (antes vs depois)

---
## 1. Instalação de Dependências

In [ ]:
!pip install transformers torch accelerate bitsandbytes peft trl datasets -q

---
## 2. Configuração do Ambiente
> ⚠️ Garanta que a GPU está ativa: **Ambiente de execução → Alterar ambiente de execução → T4 GPU**

In [ ]:
import os
import torch

os.chdir('/content')

print('GPU disponível:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))

---
## 3. Carregamento do Modelo Base (Phi-3 Mini)
Usamos **quantização 4-bit** para reduzir o uso de memória da GPU.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Configuração da quantização 4-bit
quantizacao_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type='nf4',
)

model_name = 'microsoft/Phi-3-mini-4k-instruct'

print('Baixando tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(model_name)

print('Baixando modelo com quantização 4-bit...')
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantizacao_config,
    device_map='auto',
)

print('Modelo carregado com sucesso!')
print(f'Memória da GPU usada: {torch.cuda.memory_allocated() / 1e9:.2f} GB')

---
## 4. Teste do Modelo Base (Antes do Fine-Tuning)
Vamos ver como o modelo responde **antes** do treinamento, para comparar depois.

In [ ]:
def conversar_base(pergunta: str) -> str:
    """Gera resposta usando o modelo base (sem fine-tuning)."""
    messages = [
        {"role": "system", "content": "Você é um assistente especializado em ensino de programação e Vibe Coding. Responda de forma clara e didática."},
        {"role": "user", "content": pergunta}
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True
        )

    resposta = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return resposta


pergunta_teste = 'Como funciona um loop for em Python? Me dá um exemplo simples.'
print(f'Pergunta: {pergunta_teste}\n')
print('=' * 50)
print('RESPOSTA DO MODELO BASE:\n')
print(conversar_base(pergunta_teste))

---
## 5. Preparação do Dataset
> 📁 Faça upload dos arquivos `dataset_exemplo.py`, `dataset_completo.py` e `preparar_dataset.py` no Colab antes de rodar essa célula.

In [ ]:
# Gera dataset.json (3 exemplos base)
exec(open('/content/dataset_exemplo.py').read())

# Gera dataset_completo.json (17 exemplos)
exec(open('/content/dataset_completo.py').read())

# Combina e formata tudo no padrão do Phi-3
exec(open('/content/preparar_dataset.py').read())

---
## 6. Classe do Dataset para Treinamento

In [ ]:
import json
from torch.utils.data import Dataset


class DatasetFineTuning(Dataset):
    """Dataset formatado para o fine-tuning do Phi-3."""

    def __init__(self, caminho: str, tokenizer, max_length: int = 512):
        self.tokenizer = tokenizer
        self.max_length = max_length

        with open(caminho, 'r', encoding='utf-8') as f:
            self.dados = json.load(f)

        print(f'Dataset carregado com {len(self.dados)} exemplos.')

    def __len__(self) -> int:
        return len(self.dados)

    def __getitem__(self, idx: int) -> dict:
        exemplo = self.dados[idx]

        texto = self.tokenizer.apply_chat_template(
            exemplo['messages'],
            tokenize=False,
            add_generation_prompt=False
        )

        tokenizado = self.tokenizer(
            texto,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        input_ids = tokenizado['input_ids'].squeeze()
        attention_mask = tokenizado['attention_mask'].squeeze()

        labels = input_ids.clone()
        labels[attention_mask == 0] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
        }


dataset = DatasetFineTuning('/content/dataset_final.json', tokenizer)
print(f'Dataset pronto! Exemplos: {len(dataset)}')

---
## 7. Configuração do LoRA
**LoRA (Low-Rank Adaptation)** permite treinar apenas **0.25%** dos parâmetros do modelo, tornando o fine-tuning possível mesmo em GPUs gratuitas.

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

lora_config = LoraConfig(
    r=16,                                    # Dimensão das matrizes LoRA
    lora_alpha=32,                           # Escala de aprendizado
    target_modules=['qkv_proj', 'o_proj'],   # Camadas ajustadas no Phi-3
    lora_dropout=0.05,                       # Regularização
    bias='none',
    task_type='CAUSAL_LM',
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

---
## 8. Treinamento (Fine-Tuning)
O modelo vai passar pelo dataset 3 vezes (epochs). O **Loss** deve diminuir a cada epoch, indicando aprendizado.

In [ ]:
from torch.utils.data import DataLoader
from torch.optim import AdamW

# Configurações do treinamento
EPOCHS = 3
BATCH_SIZE = 2
LEARNING_RATE = 2e-4

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
model.train()

print('=== Iniciando Treinamento ===\n')

for epoch in range(EPOCHS):
    total_loss = 0

    for batch_idx, batch in enumerate(dataloader):
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        labels = batch['labels'].to(model.device)

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss

        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        print(f'  Epoch {epoch+1}/{EPOCHS} | Batch {batch_idx+1}/{len(dataloader)} | Loss: {loss.item():.4f}')

    media_loss = total_loss / len(dataloader)
    print(f'\n  Epoch {epoch+1} finalizada | Loss média: {media_loss:.4f}\n')

print('=== Treinamento Concluído! ===')

---
## 9. Salvamento do Modelo Treinado

In [ ]:
import os

DIRETORIO_MODELO = '/content/modelo_treinado'
os.makedirs(DIRETORIO_MODELO, exist_ok=True)

model.save_pretrained(DIRETORIO_MODELO)
tokenizer.save_pretrained(DIRETORIO_MODELO)

print(f'Modelo salvo em: {DIRETORIO_MODELO}')
print('Arquivos salvos:', os.listdir(DIRETORIO_MODELO))

---
## 10. Teste do Modelo Treinado (Após Fine-Tuning)
Comparando as respostas **antes** e **depois** do treinamento.

In [ ]:
def conversar_treinado(pergunta: str) -> str:
    """Gera resposta usando o modelo após fine-tuning."""
    messages = [
        {
            'role': 'system',
            'content': (
                'Você é um assistente especializado em ensino de programação e Vibe Coding. '
                'Sempre responda com código bem estruturado, organizado e seguindo boas práticas: '
                'type hints, docstrings, separação de responsabilidades e comentários explicativos.'
            )
        },
        {'role': 'user', 'content': pergunta}
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True
        )

    resposta = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return resposta


# Teste 1: Loop for
pergunta1 = 'Como funciona um loop for em Python? Me dá um exemplo simples.'
print(f'Pergunta: {pergunta1}\n')
print('=' * 50)
print('RESPOSTA DO MODELO TREINADO:\n')
print(conversar_treinado(pergunta1))

In [ ]:
# Teste 2: Classes
pergunta2 = 'Como criar uma classe em Python pra representar um produto?'
print(f'Pergunta: {pergunta2}\n')
print('=' * 50)
print('RESPOSTA DO MODELO TREINADO:\n')
print(conversar_treinado(pergunta2))

In [ ]:
# Teste 3: Vibe Coding
pergunta3 = 'Como eu uso IA pra programar de forma organizada?'
print(f'Pergunta: {pergunta3}\n')
print('=' * 50)
print('RESPOSTA DO MODELO TREINADO:\n')
print(conversar_treinado(pergunta3))

---
## ✅ Projeto Concluído!

### Resumo do que foi feito:
- Modelo base **Phi-3 Mini** carregado com quantização 4-bit
- Dataset customizado com **20 exemplos** focados em arquitetura e boas práticas
- Fine-tuning usando **LoRA** (apenas 0.25% dos parâmetros treinados)
- Modelo treinado em **3 epochs** com Loss diminuindo consistentemente
- Resultados validados: modelo passa a responder com código organizado, type hints e docstrings

### Tecnologias usadas:
- Python | PyTorch | Hugging Face Transformers | PEFT (LoRA) | Google Colab